In [ ]:
import requests
from PIL import Image
import io
import time
import threading

def get_frames(url, num_frames):
    frame_count = 0
    response = requests.get(url, stream=True)
    
    bytes_buffer = b''
    frames = []
    for chunk in response.iter_content(chunk_size=1024):
        bytes_buffer += chunk
        a = bytes_buffer.find(b'\xff\xd8')
        b = bytes_buffer.find(b'\xff\xd9')
        if a != -1 and b != -1:
            jpg = bytes_buffer[a:b+2]
            bytes_buffer = bytes_buffer[b+2:]
            image = Image.open(io.BytesIO(jpg))
            
            frame_count += 1
            frames.append(image)
            
            if frame_count == num_frames:
                return frames
    
    return frames

def stop_streaming(url):
    response = requests.post(url)
    return response.json()

def stop_streaming_in_background(url):
    # Run stop_streaming in a separate thread
    threading.Thread(target=stop_streaming, args=(url,)).start()

# Set up the URLs
base_url = "http://localhost:3000"  # Change this to your server's URL
video_feed_url = f"{base_url}/video_feed"
stop_stream_url = f"{base_url}/stop_stream"

# Set the number of frames to capture
num_frames = 1

# Start timing
start_time = time.time()

# Capture frames
frames = get_frames(video_feed_url, num_frames)

# Stop timing
end_time = time.time()

# Calculate duration
duration = end_time - start_time

# Print results
if frames:
    print(f"Successfully captured {len(frames)} frames")
    for i, frame in enumerate(frames, 1):
        frame.save(f"captured_frame_{i}.png")
    print(f"Frames saved as PNG files.")
else:
    print("Failed to capture frames")

print(f"Time taken: {duration:.2f} seconds")

# This will print immediately without waiting for stop_streaming to finish
print("Client operation completed")


# Print the server's response
response = stop_streaming(stop_stream_url)
print(f"Server response: {response}")

Successfully captured 1 frames
Frames saved as PNG files.
Time taken: 0.14 seconds


In [4]:
import cv2
import time
import threading

time.sleep(2)

url = "http://localhost:3000/video_feed"


# Initialize VideoCapture object
cap = cv2.VideoCapture(url)

# Get the default frame size
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output00.avi', fourcc, 10, (frame_width, frame_height))

# Function to continuously read frames and write them to the output video
def read_frames():
    start_time = time.time()
    frame_count = 0
    while frame_count < 100:
        ret, frame = cap.read()
        if not ret:
            print("Failed to read frame.")
            break
        frame_count += 1
        out.write(frame)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Time taken to get frames:", elapsed_time, "seconds")
    print("Frame count:", frame_count)

# Start reading frames in a separate thread
frame_thread = threading.Thread(target=read_frames)
frame_thread.start()

# Wait for the frame thread to finish
frame_thread.join()

# Release everything when done
cap.release()
out.release()

Time taken to get frames: 12.718081951141357 seconds
Frame count: 100
